**THIS FOLLOWS THE STEPS IN THE TUTORIAL FOUND ON https://www.tensorflow.org/tutorials/structured_data/time_series**

# Imports

In [1]:

import pandas as pd
import numpy as np
import tensorflow as tf

# Load datasets

In [2]:
import data_func.read_data as read_data
dataframes = read_data.get_training_data()
X_frames_train = dataframes[0]
Y_frames_train = dataframes[1]
X_frames_test = read_data.get_test_data()

## Data Analysis

In [3]:
XA = X_frames_train[0]
XB = X_frames_train[1]
XC = X_frames_train[2]

### A-set

In [10]:
XA.describe().transpose()

,count,mean,min,25%,50%,75%,max,std
date_forecast,136245,2021-05-13 14:26:20.055048,2019-06-02 22:00:00,2020-05-22 17:15:00,2021-05-12 12:30:00,2022-05-02 07:45:00,2023-04-30 23:45:00,NaN
absolute_humidity_2m:gm3,136245.0,6.322311,0.7,4.2,5.8,8.3,17.5,2.800328
air_density_2m:kgm3,136245.0,1.25328,1.145,1.228,1.251,1.277,1.427,0.037221
ceiling_height_agl:m,136245.0,2759.622803,27.799999,1283.0,1953.199951,3383.5,12328.5,2338.483643
clear_sky_energy_1h:J,136245.0,552667.1875,0.0,0.0,15320.400391,899888.8125,3005720.25,850335.1875
clear_sky_rad:W,136245.0,153.518524,0.0,0.0,0.0,250.399994,836.400024,236.805893
cloud_base_agl:m,136245.0,1673.841309,27.799999,611.799988,1135.0,1954.800049,11688.700195,1742.884155
dew_or_rime:idx,136245.0,0.020669,-1.0,0.0,0.0,0.0,1.0,0.19472
dew_point_2m:K,136245.0,275.971832,250.899994,271.299988,276.0,281.299988,293.799988,6.893843
diffuse_rad:W,136245.0,42.069656,0.0,0.0,0.0,70.599998,345.700012,63.097115


- missing values:
    - `snow_density`: 
        - Looking at the snow density we see that the min = max. It also have a lot of null values so this might tell us wether or not there is snow. We can try to fill all the missing snow values with 0. 
    - `ceiling_height`
        - A bit trickier. The missing values here are much lower, trying to fill null values with median might be a good start
    - `cloud_base_agl`
        - doing same as for ceilingheight
- constant values
    - `elevation`, `snowdrift` has a variance of 0 so it does not tell our model anything.

In [5]:
XA['snow_density:kgm3'] = XA['snow_density:kgm3'].fillna(0.0) # filling zero for missing values
XA['snow_density:kgm3'].describe()

count    136245.000000
mean          8.314067
std          44.826416
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max         250.000000
Name: snow_density:kgm3, dtype: float64

In [6]:
med = XA['ceiling_height_agl:m'].median()
XA['ceiling_height_agl:m'] = XA['ceiling_height_agl:m'].fillna(med)
XA['ceiling_height_agl:m'].describe()

count    136245.000000
mean       2759.622803
std        2338.483643
min          27.799999
25%        1283.000000
50%        1953.199951
75%        3383.500000
max       12328.500000
Name: ceiling_height_agl:m, dtype: float64

In [7]:
med = XA['cloud_base_agl:m'].median()
XA['cloud_base_agl:m'] = XA['cloud_base_agl:m'].fillna(med)
XA['cloud_base_agl:m'].describe()

count    136245.000000
mean       1673.841309
std        1742.884155
min          27.799999
25%         611.799988
50%        1135.000000
75%        1954.800049
max       11688.700195
Name: cloud_base_agl:m, dtype: float64

# Data clean up

In [3]:
def data_allign(x_train, y_train):

  y_train.dropna(inplace=True)
  combined_data = pd.merge(x_train, y_train, left_on='date_forecast', right_on='time')
  y_train = combined_data['pv_measurement']

  if 'time' and 'pv_measurement' in combined_data.columns:
    combined_data.drop(columns=['time', 'pv_measurement'], inplace=True)
    
  return combined_data, y_train

import data_func.aggregation as data_agg

for i in range(len(X_frames_train)):
    X_frames_train[i] = data_agg.gen_agg(X_frames_train[i], 'mean')
    X_frames_train[i], Y_frames_train[i] = data_allign(X_frames_train[i], Y_frames_train[i])


for j in range(len(X_frames_test)):
    X_frames_test[j] = data_agg.gen_agg(X_frames_test[j], 'mean')

Column: ceiling_height_agl:m has 3919 NULL values
Column: cloud_base_agl:m has 2094 NULL values
Column: snow_density:kgm3 has 15769 NULL values
Column: ceiling_height_agl:m has 22247 NULL values
Column: cloud_base_agl:m has 8066 NULL values
Column: snow_density:kgm3 has 115945 NULL values


Based on output we choose to drop `snow_density:kgm3`, `ceiling_height_agl:m`, `cloud_base_agl:m`

```
X_ESTIMATED
Column: ceiling_height_agl:m has 3919 NULL values
Column: cloud_base_agl:m has 2094 NULL values
Column: snow_density:kgm3 has 15769 NULL values

X_OBSERVED:
Column: ceiling_height_agl:m has 22247 NULL values
Column: cloud_base_agl:m has 8066 NULL values
Column: snow_density:kgm3 has 115945 NULL values

```

In [4]:
""" for column in X_train_estimated_a.columns:
    null_c =  X_train_estimated_a[column].value_counts()[0]
    if null_c > 0: 
        print(f'Column: {column} has {null_c} 0 values')

for column in X_train_observed_a.columns:
    null_c = X_train_observed_a[column].value_counts()[0]
    if null_c > 0: 
        print(f'Column: {column} has {null_c} 0 values') """

" for column in X_train_estimated_a.columns:\n    null_c =  X_train_estimated_a[column].value_counts()[0]\n    if null_c > 0: \n        print(f'Column: {column} has {null_c} 0 values')\n\nfor column in X_train_observed_a.columns:\n    null_c = X_train_observed_a[column].value_counts()[0]\n    if null_c > 0: \n        print(f'Column: {column} has {null_c} 0 values') "

In [5]:

# Data set A, B and C clean up
binary_featurenames = ['is_day:idx', 'is_in_shadow:idx'] # ONLY FOR REMOVAL IN QUICK TEST
categorical_featurenames = ['dew_or_rime:idx', 'precip_type_5min:idx', 'snow_drift:idx']
has_null = ['ceiling_height_agl:m', 'cloud_base_agl:m', 'snow_density:kgm3']

def data_clean_up(x_train_est, x_train_observe, y_train):

  if 'date_calc' in x_train_est.columns:
    x_train_est.drop(columns="date_calc", inplace=True)

  x_train = pd.concat([x_train_observe, x_train_est])

  # Group the rows into blocks of 4 and apply the aggregation function
  agg_func = {col: 'mean' for col in x_train.columns[1:]}
  X_train_downscaled = x_train.groupby(x_train.index // 4).agg({**{'date_forecast': 'first'}, **agg_func})

  y_train.dropna(inplace=True)
  combined_data = pd.merge(X_train_downscaled, y_train, left_on='date_forecast', right_on='time')
  #combined_data.dropna(inplace=True)
  y_train = combined_data[['pv_measurement', 'date_forecast']]

  if 'time' and 'pv_measurement' in combined_data.columns:
    combined_data.drop(columns=['time', 'pv_measurement'], inplace=True)
    combined_data.drop(columns=binary_featurenames, inplace=True)
    combined_data.drop(columns=categorical_featurenames, inplace=True)
    combined_data.drop(columns=has_null, inplace=True)

  return combined_data, y_train

def count_null_in_column(df: pd.DataFrame, column_name: str):
  return df[column_name].value_counts(None)

x_train_a, y_train_a = data_clean_up(X_train_estimated_a, X_train_observed_a, train_targets_a)
x_train_b, y_train_b = data_clean_up(X_train_estimated_b, X_train_observed_b, train_targets_b)
x_train_c, y_train_c = data_clean_up(X_train_estimated_c, X_train_observed_c, train_targets_c)

print(len(x_train_a))


def data_clean_up_test(x_test_est):

  x_test_est.drop(columns=binary_featurenames, inplace=True)
  x_test_est.drop(columns=categorical_featurenames, inplace=True)
  x_test_est.drop(columns=has_null, inplace=True)

  # Group the rows into blocks of 4 and apply the aggregation function
  agg_func = {col: 'mean' for col in x_test_est.columns[1:]}
  X_test_downscaled = x_test_est.groupby(x_test_est.index // 4).agg({**{'date_forecast': 'first'}, **agg_func})
  

  """
  if 'date_forecast' in X_test_downscaled.columns:
    X_test_downscaled.drop(columns="date_forecast", inplace=True)
  """
  return X_test_downscaled

X_test_estimated_a = data_clean_up_test(X_test_estimated_a)
X_test_estimated_b = data_clean_up_test(X_test_estimated_b)
X_test_estimated_c = data_clean_up_test(X_test_estimated_c)



29667


Keeping this average for now since its the best aggreagation we have at this point. However i do believe that using avg or mean together with delta (total change within the hour) might be a better solution. Also some columns might need other aggregations than avg. 

# Feature engineering

Need to tell the model something about how time seasonality works. E.g. night and day, as well as yearly seasonality. 

In [6]:
def get_unixtime(datetime: pd.Series) -> pd.Series:
    unixtime = (datetime - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
    return unixtime

xat_datetime = x_train_a['date_forecast']
xat_unixtime = get_unixtime(xat_datetime)

## We now need functions for assigning daily and yearly cycles (described in datanalysis docu on Peter branch)
# plus 2 avoids 0 and negative values
day = 24*60*60
year = (365.2425)*day

def sinus_day(unix_time):
    return 2 + np.sin(unix_time * (2 * np.pi / day)) # since it is seconds since 1.1.1970 we divide by seconds in a day to get seasonal changes throughout the dat

def sinus_year(unix_time):
    return 2+ np.sin(unix_time * (2 * np.pi / year))

def cosinus_day(unix_time):
    return 2+np.cos(unix_time * (2 * np.pi / day))

def cosinus_year(unix_time):
    return 2+np.cos(unix_time * (2 * np.pi / year))

# function for returning two series with the daily cycles (sine and cosine)
def get_daycycle(unixtime: pd.Series) -> (pd.Series, pd.Series):
    sinus_daytime = unixtime.apply(sinus_day)
    sinus_daytime = sinus_daytime.rename('sinus_day') 
    cosinus_daytime = unixtime.apply(cosinus_day)
    cosinus_daytime = cosinus_daytime.rename('cosine_day')
    return sinus_daytime, cosinus_daytime

# Function for returning two series with the yearly cycles
def get_yearcycle(unixtime: pd.Series) -> (pd.Series, pd.Series):
    sinus_yeartime = unixtime.apply(sinus_year)
    sinus_yeartime = sinus_yeartime.rename('sinus_year')
    cosinus_yeartime = unixtime.apply(cosinus_year)
    cosinus_yeartime = cosinus_yeartime.rename('cosinus_year')
    return sinus_yeartime, cosinus_yeartime

xat_day_sin, xat_day_cos = get_daycycle(xat_unixtime)
xat_year_sin, xat_year_cos = get_yearcycle(xat_unixtime)

xta_feat = x_train_a.join([xat_day_sin, xat_day_cos, xat_year_sin, xat_year_cos])
xta_feat.drop(columns=['date_forecast'], inplace=True)
print(len(xta_feat))

29667


Here we need to do some extra feature engineering with regards to different datatypes in the dataset. A lot of this is explained in the TensorFlow documentation https://www.tensorflow.org/tutorials/load_data/pandas_dataframe. 

### Setting categorical and binary values as int

Tensorflow usuallt interprets data as float32. However for categorical and binary data we want it to be interpreted as integers. Therefore we set these specific columns as datatype int, and the rest as datatype float. Here we utilize the dictionary way of creating a model as opposed to the numpy array way. This is because our data is NOT homogenous. 

In [7]:
target = y_train_a
target.drop(columns='date_forecast', inplace=True)
xta_numeric_dict_ds = tf.data.Dataset.from_tensor_slices((dict(xta_feat), target))
for row in xta_numeric_dict_ds.take(3):
  print(row)

({'absolute_humidity_2m:gm3': <tf.Tensor: shape=(), dtype=float32, numpy=8.025>, 'air_density_2m:kgm3': <tf.Tensor: shape=(), dtype=float32, numpy=1.230625>, 'clear_sky_energy_1h:J': <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, 'clear_sky_rad:W': <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, 'dew_point_2m:K': <tf.Tensor: shape=(), dtype=float32, numpy=280.7875>, 'diffuse_rad:W': <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, 'diffuse_rad_1h:J': <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, 'direct_rad:W': <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, 'direct_rad_1h:J': <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, 'effective_cloud_cover:p': <tf.Tensor: shape=(), dtype=float32, numpy=99.5375>, 'elevation:m': <tf.Tensor: shape=(), dtype=float32, numpy=6.0>, 'fresh_snow_12h:cm': <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, 'fresh_snow_1h:cm': <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, 'fresh_snow_24h:cm': <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, 'fresh_

/var/folders/94/wm45lcfj7sv746_0rnyvdk840000gn/T/ipykernel_33968/1542623841.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target.drop(columns='date_forecast', inplace=True)


### Preprocessing pipeline (not in quick)

In [8]:
binary_featurenames = ['is_day:idx', 'is_in_shadow:idx']
categorical_featurenames = ['dew_or_rime:idx', 'precip_type_5min:idx', 'snow_drift:idx']

inputs = {}
for name, column in xta_feat.items():
  dtype = tf.float32
  inputs[name] = tf.keras.Input(shape=(), name=name, dtype=dtype)
inputs

{'absolute_humidity_2m:gm3': <KerasTensor: shape=(None,) dtype=float32 (created by layer 'absolute_humidity_2m:gm3')>,
 'air_density_2m:kgm3': <KerasTensor: shape=(None,) dtype=float32 (created by layer 'air_density_2m:kgm3')>,
 'clear_sky_energy_1h:J': <KerasTensor: shape=(None,) dtype=float32 (created by layer 'clear_sky_energy_1h:J')>,
 'clear_sky_rad:W': <KerasTensor: shape=(None,) dtype=float32 (created by layer 'clear_sky_rad:W')>,
 'dew_point_2m:K': <KerasTensor: shape=(None,) dtype=float32 (created by layer 'dew_point_2m:K')>,
 'diffuse_rad:W': <KerasTensor: shape=(None,) dtype=float32 (created by layer 'diffuse_rad:W')>,
 'diffuse_rad_1h:J': <KerasTensor: shape=(None,) dtype=float32 (created by layer 'diffuse_rad_1h:J')>,
 'direct_rad:W': <KerasTensor: shape=(None,) dtype=float32 (created by layer 'direct_rad:W')>,
 'direct_rad_1h:J': <KerasTensor: shape=(None,) dtype=float32 (created by layer 'direct_rad_1h:J')>,
 'effective_cloud_cover:p': <KerasTensor: shape=(None,) dtype=f

In [9]:
""" preprocessed = []

for name in binary_featurenames:
  inp = inputs[name]
  inp = inp[:, tf.newaxis]
  float_value = tf.cast(inp, tf.float32)
  preprocessed.append(float_value)

preprocessed """

' preprocessed = []\n\nfor name in binary_featurenames:\n  inp = inputs[name]\n  inp = inp[:, tf.newaxis]\n  float_value = tf.cast(inp, tf.float32)\n  preprocessed.append(float_value)\n\npreprocessed '

In [10]:
preprocessed = []
def stack_dict(inputs, fun=tf.stack):
    values = []
    for key in sorted(inputs.keys()):
      values.append(tf.cast(inputs[key], tf.float32))

    return fun(values, axis=-1)

# getting numerical features
numeric_feature_names = []
for col in xta_feat.columns:
   if col not in binary_featurenames and col not in categorical_featurenames:
    numeric_feature_names.append(col)
numeric_features = xta_feat[numeric_feature_names]

normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(stack_dict(dict(numeric_features)))

# creating new dict to send to normalizer and append to preprocessed
numeric_inputs = {}
for name in numeric_feature_names:
  numeric_inputs[name]=inputs[name]

numeric_inputs = stack_dict(numeric_inputs)
numeric_normalized = normalizer(numeric_inputs)

preprocessed.append(numeric_normalized)

preprocessed

[<KerasTensor: shape=(None, 41) dtype=float32 (created by layer 'normalization')>]

In [11]:
""" for name in categorical_featurenames:
  vocab = sorted(set(xta_feat[name]))
  print(f'name: {name}')
  print(f'vocab: {vocab}\n')

  if type(vocab[0]) is str:
    lookup = tf.keras.layers.StringLookup(vocabulary=vocab, output_mode='one_hot')
  else:
    lookup = tf.keras.layers.IntegerLookup(vocabulary=vocab, output_mode='one_hot')

  x = inputs[name][:, tf.newaxis]
  x = lookup(x)
  preprocessed.append(x) """

" for name in categorical_featurenames:\n  vocab = sorted(set(xta_feat[name]))\n  print(f'name: {name}')\n  print(f'vocab: {vocab}\n')\n\n  if type(vocab[0]) is str:\n    lookup = tf.keras.layers.StringLookup(vocabulary=vocab, output_mode='one_hot')\n  else:\n    lookup = tf.keras.layers.IntegerLookup(vocabulary=vocab, output_mode='one_hot')\n\n  x = inputs[name][:, tf.newaxis]\n  x = lookup(x)\n  preprocessed.append(x) "

In [12]:
preprocesssed_result = tf.concat(preprocessed, axis=-1)
preprocesssed_result

<KerasTensor: shape=(None, 41) dtype=float32 (created by layer 'tf.identity')>

In [13]:
preprocessor = tf.keras.Model(inputs, preprocesssed_result)
tf.keras.utils.plot_model(preprocessor, rankdir="LR", show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [32]:
body = tf.keras.Sequential([
  tf.keras.layers.Dense(5),
  #tf.keras.layers.Dense(10, activation='relu'),
  tf.keras.layers.Dense(1)
])

In [28]:
x = preprocessor(inputs)

In [33]:
result = body(x)

In [34]:
model = tf.keras.Model(inputs, result)

model.compile(optimizer='adam',
                loss=tf.keras.losses.MeanSquaredError(reduction="auto", name="mean_squared_error"),
                metrics=['accuracy'])

# Training the model

In [35]:
SHUFFLE_BUFFER = 500
BATCH_SIZE = 2

model.fit(dict(xta_feat), target, epochs=5, batch_size=BATCH_SIZE)


Epoch 1/5
14834/14834 [==============================] - 13s 875us/step - loss: 987972.9375 - accuracy: 0.4093
Epoch 2/5
14834/14834 [==============================] - 13s 870us/step - loss: 393057.1562 - accuracy: 0.3746
Epoch 3/5
14834/14834 [==============================] - 13s 894us/step - loss: 320507.0000 - accuracy: 0.2947
Epoch 4/5
14834/14834 [==============================] - 14s 968us/step - loss: 305607.3750 - accuracy: 0.2630
Epoch 5/5
14834/14834 [==============================] - 14s 922us/step - loss: 299016.5625 - accuracy: 0.2570


# Make predictions

In [ ]:
# Do some more stuff

# Evaluate prediction

In [ ]:
# Evaluate the model

y_pred = []

# Create submission

In [ ]:
y_test_pred = y_pred

test = pd.read_csv('../data/test.csv')
test['prediction'] = y_test_pred
sample_submission = pd.read_csv('../data/sample_submission.csv')
submission = sample_submission[['id']].merge(test[['id', 'prediction']], on='id', how='left')
submission.to_csv('submission.csv', index=False)